# SleepTalk

## Categories:

| Category       | Label  |
|----------------|--------|
| Talk           | SPEECH |
| Snoring        | SNORE  |
| Sighs          | SIGH   |
| Farts          | FART   |
| Loud breathing | BREATH |
| Cough          | COUGH  |

## Setup

In [7]:
# define classes TODO: define depending on training data

classes = ["SPEECH", "SNORE", "SIGH", "FART", "BREATH", "COUGH"]
no_class = "NONE"

## Training data collection



In [4]:
import urllib.request
import tarfile
import os
import shutil

google_data_dir = "google"

In [2]:
# download google training embeddings
if os.path.exists(google_data_dir):
    shutil.rmtree(google_data_dir)
    
print("Downloading training features...")
with urllib.request.urlopen("https://storage.googleapis.com/eu_audioset/youtube_corpus/v1/features/features.tar.gz") as response, tarfile.open(fileobj=response, mode='r|gz') as targz:
    targz.extractall(filter='tar')
    os.rename("audioset_v1_embeddings", google_data_dir)

print("done")

In [5]:
google_label_file = f"{google_data_dir}/labels.csv"
if not os.path.exists(google_label_file):
    urllib.request.urlretrieve("https://storage.googleapis.com/us_audioset/youtube_corpus/v1/csv/class_labels_indices.csv", google_label_file)


In [8]:
synonyms = {
    classes[0]: ["Speech", "Shout", "Whispering"], # SPEECH
    classes[1]: ["Snoring"], # SNORE
    classes[2]: ["Sigh", "Groan", "Grunt"], # SIGH
    classes[3]: ["Stomach rumble", "Fart"], # FART
    classes[4]: ["Yawn", "Sniff", "Wheeze", "Gasp", "Pant", "Snort"], # BREATH
    classes[5]: ["Cough", "Sneeze"], # COUGH
    no_class: ["Chewing, mastication", "Biting", "Burping, eructation", "Bang", "Slap, smack", "Whack, thwack", "Smash, crash", "Knock", "Tap", "Flap", "Vehicle", "Alarm", "Door", "Thunderstorm", "Wind", "Water", "Noise"]
}

switched_synonyms = {}
for (key, entry) in synonyms.items():
    for synonym in entry:
        switched_synonyms[synonym] = key

synonyms = switched_synonyms

In [9]:
import csv

label_to_class = {}
with open(google_label_file, newline='') as label_file:
    reader = csv.reader(label_file)
    for row in filter(lambda r: r[2] in synonyms.keys(), reader):
        label_to_class[int(row[0])] = synonyms[row[2]]


In [24]:
import tensorflow as tf

def get_sequence_examples(filename):
    iterator = tf.compat.v1.io.tf_record_iterator(path=filename)
    
    result = []
    for string_record in iterator:
        example = tf.train.SequenceExample()
        example.ParseFromString(string_record)
        result.append(example)
    return result
    

def get_labels(sequence):
    return sequence.context.feature["labels"].int64_list.value


def get_label_vector(labels: list[int]):
    class_names = {label_to_class[label] for label in labels}
    label_data = [0 for _ in range(len(classes))]
    for class_name in class_names:
        if class_name != no_class:
            label_data[classes.index(class_name)] = 1
    return label_data == [0 for _ in range(len(classes))], label_data
    
google_embeddings_class = []
google_labels_class = []
google_embeddings_no_class = []
google_labels_no_class = []



balanced_train_dir = f"{google_data_dir}/bal_train"
filenames = [f"{balanced_train_dir}/{file}" for file in os.listdir(balanced_train_dir)]

for filename in filenames:
    for example in get_sequence_examples(filename):
        labels = get_labels(example)
        if label_to_class.keys().isdisjoint(labels):
            continue
        no_class, label_vector = get_label_vector(labels)
        # TODO: embeddings extrahieren
        
        if no_class:
            google_labels_no_class.append(label_vector)
            # TODO google_embeddings_no_class.append(embeddings)
        else:
            google_labels_class.append(label_vector)
            # TODO google_embeddings_class.append(embeddings)
        break # TODO: entfernen
    break # TODO: entfernen
    
    
    
    

True [0, 0, 0, 0, 0, 0]


In [ ]:
# TODO: remove google data

## Classification model definition

In [3]:
from tf_keras import Sequential
from tensorflow_hub import KerasLayer
from tf_keras.layers import Conv2D, MaxPooling2D, Flatten, Dropout, Dense

classification_model = Sequential()

# adding layers for classification
classification_model.add(Conv2D(64, kernel_size=5, strides=1, padding="Same", activation="relu"))
classification_model.add(MaxPooling2D(padding="same"))

classification_model.add(Conv2D(128, kernel_size=5, strides=1, padding="same", activation="relu"))
classification_model.add(MaxPooling2D(padding="same"))
classification_model.add(Dropout(0.3))

classification_model.add(Flatten())

classification_model.add(Dense(256, activation="relu"))
classification_model.add(Dropout(0.3))

classification_model.add(Dense(512, activation="relu"))
classification_model.add(Dropout(0.3))

classification_model.add(Dense(len(classes), activation="sigmoid"))


2024-07-22 21:33:08.685618: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-07-22 21:33:08.738918: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-07-22 21:33:08.789849: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-07-22 21:33:08.832429: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-07-22 21:33:08.844710: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-07-22 21:33:08.930615: I tensorflow/core/platform/cpu_feature_gu

In [4]:
classification_model.compile(optimizer="adam", loss="categorical_crossentropy", metrics=["accuracy"])

## Model training

In [5]:
# TODO

## Model export

In [14]:
import tensorflow as tf

# adding Googles VGGish model for sound embeddings
# TODO: look into licensing
vggish = KerasLayer("https://www.kaggle.com/models/google/vggish/TensorFlow2/vggish/1")

# adding classification model
model = Sequential([
    vggish,
    classification_model
])

converter = tf.lite.TFLiteConverter.from_keras_model(model)
tflite_model = converter.convert()

with tf.io.gfile.GFile("model.tflite", "wb") as file:
    file.write(tflite_model)

TypeError: Exception encountered when calling layer 'keras_layer_8' (type KerasLayer).

Binding inputs to tf.function failed due to `Can not cast TensorSpec(shape=(4, 5), dtype=tf.float32, name=None) to TensorSpec(shape=(None,), dtype=tf.float32, name=None)`. Received args: ([[<tf.Tensor: shape=(), dtype=float32, numpy=0.0>, <tf.Tensor: shape=(), dtype=float32, numpy=0.0>, <tf.Tensor: shape=(), dtype=float32, numpy=0.0>, <tf.Tensor: shape=(), dtype=float32, numpy=0.0>, <tf.Tensor: shape=(), dtype=float32, numpy=0.0>], [<tf.Tensor: shape=(), dtype=float32, numpy=0.0>, <tf.Tensor: shape=(), dtype=float32, numpy=0.0>, <tf.Tensor: shape=(), dtype=float32, numpy=0.0>, <tf.Tensor: shape=(), dtype=float32, numpy=0.0>, <tf.Tensor: shape=(), dtype=float32, numpy=0.0>], [<tf.Tensor: shape=(), dtype=float32, numpy=0.0>, <tf.Tensor: shape=(), dtype=float32, numpy=0.0>, <tf.Tensor: shape=(), dtype=float32, numpy=0.0>, <tf.Tensor: shape=(), dtype=float32, numpy=0.0>, <tf.Tensor: shape=(), dtype=floa